In this project, we want to use SQL to make a database of students' schedules and use a database of major requirements for data science majors to make a schdule builder. We want to take advantage of relational databases here. Let's start by structuring out our databases. 

In [103]:
from sqlalchemy import create_engine
import psycopg2

db_params = {
    "dbname": "postgres",
    "user": "madikloberdanz",
    "password": "3141",
    "host": "localhost",
    "port": "5433"
}

# Create a SQLAlchemy engine to connect to the PostgreSQL database
engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}")

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(**db_params)

# Create a cursor object for executing SQL queries
c = conn.cursor()


# Drop the table if it exists
# c.execute("DROP TABLE IF EXISTS Courses")

# Courses Table
c.execute("""CREATE TABLE IF NOT EXISTS Courses (
    course_id SERIAL PRIMARY KEY,
    course_name TEXT,
    prereq_id INT[],
    term_offered TEXT[],
    time_offered TEXT[])""")


# Establish relations
# c.execute("""ALTER TABLE Courses
# DROP CONSTRAINT IF EXISTS fk_prerequisite;

# ALTER TABLE Courses
# ADD CONSTRAINT fk_prerequisite
# FOREIGN KEY (prereq_id)
# REFERENCES Courses (course_id)
# NOT VALID;

# """)


Now that we've established our two tables and the relationships between them, let's make sure we are able to insert students, pull information from a student, update a student's information (for example, if they complete a class or change their schedule), and delete a student (if they drop the major).

In [104]:
# Add a student
def add_student(student_id, student_name, years_left, earned_credits, ongoing_credits):
    query = """INSERT INTO Students (student_id, student_name, years_left, earned_credits, ongoing_credits)
               VALUES (%s, %s, %s, %s, %s)"""
    values = (student_id, student_name, years_left, earned_credits, ongoing_credits)
    c.execute(query, values)
    conn.commit()

# Get information about a student
def get_earned_credits(student_id):
    query = """SELECT 
            ARRAY(earned_credits) 
            FROM Students WHERE student_id=%s"""
    values = (student_id,)
    c.execute(query, values)
    return c.fetchall()

# Update student info
def update_courses(student_id, course_taken):
    query = """UPDATE Students
               SET earned_credits = ARRAY_APPEND(earned_credits, %s)
               SET ongoing_credits = ARRAY_REMOVE(earned_credits, %s)
               WHERE student_id = %s"""
    values = (course_taken, student_id)
    c.execute(query, values)
    conn.commit()

# Delete a student
def delete_student(student_id):
    query = """DELETE FROM Students WHERE student_id=%s"""
    values = (student_id,)
    c.execute(query, values)
    conn.commit()



Of course, we also need to make similar functions for the classes database.

In [105]:
# Add a class
def add_course(course_name, prereq_names, term_offered, time_offered):
    # Check if the course has already been added
    c.execute("SELECT course_id FROM Courses WHERE course_name = %s", (course_name,))
    existing_course = c.fetchone()
    
    if existing_course:
        print(f"Course '{course_name}' aready exists")
        return 
    
    # If there are no pre-reqs, make the pre-req ID NULL
    if prereq_names is None:
        query = """INSERT INTO Courses(course_name, prereq_id, term_offered, time_offered)
               VALUES (%s, NULL, %s, %s)"""
        values = (course_name, term_offered, time_offered)
    else:
        # Fetch the prereq_ids for the given prereq_names
        prereq_ids = []
        for prereq_name in prereq_names:
            c.execute("SELECT course_id FROM Courses WHERE course_name = %s", (prereq_name,))
            prereq_id = c.fetchone()
            if prereq_id:
                prereq_ids.append(prereq_id[0])

        # Insert the new course with multiple prereq_ids
        query = """INSERT INTO Courses(course_name, prereq_id, term_offered, time_offered)
                VALUES (%s, %s, %s, %s)"""
        values = (course_name, prereq_ids, term_offered, time_offered)


    try:
        c.execute(query, values)
        conn.commit()
        print("Course added successfully!")
    except (Exception, psycopg2.Error) as error:
        print("Error adding course:", error)


# Get all information about a course
def get_course_info(course_name):
    query = """SELECT * 
            FROM Courses WHERE course_name=%s"""
    values = (course_name,)
    c.execute(query, values)
    return c.fetchall()

# Update course info
def update_term_and_time(course_name, new_term, new_time):
    query = """UPDATE Courses
               SET term_offered=%s
               SET time_offered=%s
               WHERE course_name = %s"""
    values = (new_term, new_time, course_name)
    c.execute(query, values)
    conn.commit()

# Delete a course
def delete_course(course_name):
    query = """DELETE FROM Courses WHERE course_name=%s"""
    values = (course_name,)
    c.execute(query, values)
    conn.commit()

Now, let's actually make a funtion actaully build a schedule for a student, checking for time conflicts and pre-requisites. Right now, this function isn't very advanced and it doesn't optimize anything. Later down the line, we can make a more complicated schedule builder.

In [106]:
def make_schedule(student_id):
    query = ("""SELECT course_name
    FROM Courses
    WHERE prereq_id IS NULL
    OR prereq_id IN (
        SELECT course_id
        FROM Courses
        WHERE course_id IN (SELECT course_id FROM Courses WHERE prereq_id IS NULL)
        WHERE student_id=%s
    )""")
    values = (student_id,)
    c.execute(query, values)


In order to actually be able to use the schedule builder, we need to insert some courses from the University of Oregon's [Data Science major requirements page](https://scds.uoregon.edu/ds/undergraduate-program/major-requirements).

In [107]:
add_course("DSCI 101", None, ["Fall", "Winter"], ["11:20AM"])
add_course("DSCI 102", ["DSCI 101"], ["Winter", "Spring"], ["11:20AM"])
get_course_info("DSCI 102")

Course 'DSCI 101' aready exists
Course 'DSCI 102' aready exists


[(11, 'DSCI 102', [1], ['Winter', 'Spring'], ['11:20AM'])]

Now that we know that we can add some courses with and without pre-reqs, let's add a lot of courses at once.

In [108]:
add_course("CS 210", None, ["Fall", "Winter"], ["10:00AM"])
add_course("CS 211", ["CS 210"], ["Winter", "Spring"], ["10:00AM"])
add_course("CS 212", ["CS 210", "CS 211"], ["Fall", "Spring"], ["10:00AM"])
add_course("MATH 251", None, ["Fall", "Winter", "Spring", "Summer"], ["9:00AM", "4:00PM"])
add_course("MATH 252", ["MATH 251"], ["Fall", "Winter", "Spring", "Summer"], ["9:00AM", "4:00PM"])
add_course("MATH 341", ["MATH 252"], ["Fall", "Winter", "Summer"], ["9:00AM", "12:00PM"])
add_course("MATH 342", ["MATH 341"], ["Winter", "Spring", "Summer"], ["9:00AM", "12:00PM"])
add_course("DSCI 311", ["DSCI 102", "CS 211", "MATH 252", "MATH 342"], ["Fall"], ["11:20AM"])

Course 'CS 210' aready exists
Course 'CS 211' aready exists
Course 'CS 212' aready exists
Course 'MATH 251' aready exists
Course 'MATH 252' aready exists
Course 'MATH 341' aready exists
Course 'MATH 342' aready exists
Course 'DSCI 311' aready exists
